In [ ]:
import polars as pl
import polars as pl
import alphalens as al
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
# factor_df = pl.read_parquet ('data/alphalens/factor_df.parquet').to_pandas()
# print (f'factor_df: {factor_df} {factor_df.index}')
# price_df = pl.read_parquet ('data/alphalens/price_df.parquet').to_pandas()

import pandas as pd
factor_df = pd.read_parquet ('data/alphalens/factor_df.parquet')
# print (factor_df, factor_df.index)
print (factor_df.index)
price_df = pd.read_parquet ('data/alphalens/price_df.parquet')


In [ ]:

al_factor_data = al.utils.get_clean_factor_and_forward_returns (factor_df, price_df, quantiles=4, max_loss=0)

al_factor_data

In [ ]:

# al.performance.mean_return_by_quantile(factor_data, by_group = True, demeaned = False, group_adjust = False)
# al.performance.mean_return_by_quantile(factor_data, by_group = False, demeaned = False, group_adjust = False)

# al.tears.create_full_tear_sheet(al_factor_data, long_short=True, group_neutral = True, by_group = False)
# al.tears.create_full_tear_sheet(al_factor_data, long_short=True, group_neutral = True)
al.tears.create_full_tear_sheet(al_factor_data, long_short=False)

In [ ]:

# al.tears.create_full_tear_sheet(al_factor_data, long_short=False)

In [ ]:
al.tears.create_information_tear_sheet(al_factor_data)

In [ ]:
help (al.tears.create_information_tear_sheet)

In [ ]:
al.tears.create_returns_tear_sheet(al_factor_data, long_short=False)

In [ ]:
from alphalens import utils,performance,plotting
import matplotlib.pyplot as plt
from pandas.tseries.offsets import Day

def plot_performance(factor, prices):

    factor_data = utils.get_clean_factor_and_forward_returns(factor,prices,quantiles=20,periods=(1,7,10))
    # ic = performance.factor_information_coefficient(factor_data)
    # plotting.plot_ic_hist(ic)
    # plotting.plot_ic_ts(ic)

    # mean_ic = performance.mean_information_coefficient(factor_data,by_time="M")
    # plotting.plot_monthly_ic_heatmap(mean_ic)

    # 按quantile区分的持股平均收益（减去了总体平均值）
    mean_return_by_q = performance.mean_return_by_quantile(factor_data, by_date=True, demeaned=True)[0]    
    if mean_return_by_q.shape[0] > 1000:
        mean_return_by_q = mean_return_by_q.iloc[-1000:]
    # 按quantile画出累积持有收益
    for i in [1, 5, 10]:
        plotting.plot_cumulative_returns_by_quantile(mean_return_by_q, period=i, freq=Day())
    plt.show()

plot_performance(factor_df, price_df)

In [ ]:
mean_return_by_q = performance.mean_return_by_quantile(al_factor_data, by_date=True, demeaned=True)[0]    
# performance.mean_return_by_quantile(al_factor_data, by_date=True, demeaned=True)
mean_return_by_q

In [ ]:
help (al.performance.mean_return_by_quantile)
mean_return_by_q.columns
# mean_return_by_q['1D'].plot()
mean_return_by_q, mean_return_by_q.shape, mean_return_by_q.index

In [ ]:
# mean_return_by_q['1D'] - mean_return_by_q['5D']
for i in range (5):
    print (mean_return_by_q['1D'].iloc[i], mean_return_by_q['5D'].iloc[i], mean_return_by_q['10D'].iloc[i])

In [ ]:
import polars as pl
import numpy as np

# 创建一个只包含 0 和 -1 的 DataFrame
df = pl.DataFrame({
    'binary_col': [-1, 0, -1, 0, 0, -1, 0, -1, -1, 0]
})

# 计算偏度
skewness = df.select(pl.col('binary_col').skew()).item()

print(f"Skewness: {skewness}")

# 创建不同分布的数据来比较
df_balanced = pl.DataFrame({'balanced': [-1, 0, -1, 0, -1, 0, -1, 0, -1, 0]})
df_imbalanced = pl.DataFrame({'imbalanced': [-1, -1, -1, -1, -1, -1, -1, -1, 0, 0]})

print("Balanced distribution skewness:", df_balanced.select(pl.col('balanced').skew()).item())
print("Imbalanced distribution skewness:", df_imbalanced.select(pl.col('imbalanced').skew()).item())

# 使用 NumPy 来验证
from scipy.stats import skew

print("\nUsing NumPy/SciPy:")
print("Original data skewness:", skew(df['binary_col']))
print("Balanced data skewness:", skew(df_balanced['balanced']))
print("Imbalanced data skewness:", skew(df_imbalanced['imbalanced']))

# Calculate rolling skewness for df_balanced
window_size = 5  # You can adjust this value as needed
df_balanced = df_balanced.with_columns(
    pl.col('balanced').rolling_skew(window_size).alias('rolling_skew')
)

print("\nRolling Skewness for Balanced Distribution:")
print(df_balanced)

# If you want to plot the rolling skewness
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(df_balanced['rolling_skew'])
plt.title(f'Rolling Skewness (Window Size: {window_size})')
plt.xlabel('Index')
plt.ylabel('Skewness')
plt.grid(True)
plt.show()


In [ ]:
import polars as pl

data = pl.read_parquet('data/pure_day_data_agg_from_hour_sep29.parquet').sort ('open_time')
boris_data = pl.read_parquet('data/all_data_1d_boris.parquet').sort ('OpenTime')
# Convert OpenTime to OpenTimeDate
boris_data = boris_data.with_columns(
    pl.from_epoch("OpenTime", time_unit="ms").alias("OpenTimeDate"),
    pl.from_epoch("CloseTime", time_unit="ms").alias("CloseTimeDate")
)


# Reorder columns to move OpenTimeDate and Symbol to the front
boris_data = boris_data.select(
    ["OpenTimeDate", "Symbol"] + 
    [col for col in boris_data.columns if col not in ["OpenTimeDate", "Symbol"]]
)


boris_data

In [ ]:
SYMBOL = 'BLUEBIRDUSDT'

data.filter (pl.col ('symbol') == SYMBOL)

In [ ]:

print (data.shape, boris_data.shape)
boris_data.filter (pl.col ('Symbol') == SYMBOL)

In [ ]:
def ConvertFromBorisColumns(input_df):
    column_mapping = {
        'Symbol': 'symbol',
        
        'OpenTime': 'open_time_ms',
        'CloseTime': 'close_time_ms',
        'OpenTimeDate': 'open_time',
        'CloseTimeDate': 'close_time',
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Volume': 'volume',
        'BaseAssetVolume': 'quote_volume',
        'NumberOfTrades': 'count',
        'TakerBuyVolume': 'taker_buy_volume',
        'TakerBuyBaseAssetVolume': 'taker_buy_quote_volume',
        'column_11': 'ignore',
        
    }
    
    # Create a new DataFrame with renamed columns
    result_df = input_df.rename(column_mapping)
    
    return result_df

# Example usage:
# converted_df = ConvertFromBorisColumns(boris_data)
# print(converted_df)
new_boris_data = ConvertFromBorisColumns (boris_data)
new_boris_data

In [ ]:
new_boris_data.write_parquet ('data/all_data_1d_boris_converted.parquet')

In [ ]:
# Compare column names
boris_columns = set(new_boris_data.columns)
data_columns = set(data.columns)

print("Columns are identical:", boris_columns == data_columns)
if boris_columns != data_columns:
    print("Columns only in new_boris_data:", boris_columns - data_columns)
    print("Columns only in data:", data_columns - boris_columns)

# Compare first 10 rows
# boris_first_10 = new_boris_data.head(10)
# data_first_10 = data.head(10)

def Compare (boris_first_10, data_first_10):
    # Sort both DataFrames by 'open_time' and 'symbol' to ensure consistent ordering
    boris_first_10 = boris_first_10.sort(["open_time", "symbol"])
    data_first_10 = data_first_10.sort(["open_time", "symbol"])

    # Compare each column
    for col in boris_columns.intersection(data_columns):
        if col in ['open_time', 'close_time']:  # Compare datetime columns separately
            boris_times = boris_first_10[col].dt.timestamp()
            data_times = data_first_10[col].dt.timestamp()
            is_equal = (boris_times == data_times).all()
        else:
            is_equal = (boris_first_10[col] == data_first_10[col]).all()
        
        print(f"Column '{col}' is identical: {is_equal}")

    # Overall comparison
    all_columns_identical = all((boris_first_10[col] == data_first_10[col]).all() for col in boris_columns.intersection(data_columns))
    print("\nAll columns in first 10 rows are identical:", all_columns_identical)


In [ ]:

SYMBOL = 'NOTUSDT'
not1 = data.sort ('open_time').filter (pl.col ('symbol') == SYMBOL)
not2 = new_boris_data.sort ('open_time').filter (pl.col ('symbol') == SYMBOL)

# Find the common open_time range
common_open_times = set(not1['open_time_ms']).intersection(set(not2['open_time_ms']))

print (f'common_open_times: {common_open_times}')

# Filter both DataFrames to only include rows with common open_times
not1_common = not1.filter(pl.col('open_time_ms').is_in(common_open_times))
not2_common = not2.filter(pl.col('open_time_ms').is_in(common_open_times))

# Sort both DataFrames by open_time to ensure consistent ordering
not1_common = not1_common.sort('open_time')
not2_common = not2_common.sort('open_time')

print (not1_common.shape, not2_common.shape)




In [ ]:
import polars as pl
import pandas as pd
import numpy as np

def compare_dataframes(df1: pl.DataFrame, df2: pl.DataFrame, symbol: str, decimal_places = 4):
    # Filter DataFrames for the given symbol
    df1 = df1.filter(pl.col('symbol') == symbol)
    df2 = df2.filter(pl.col('symbol') == symbol)
    
    # Find common open_times
    common_open_times = set(df1['open_time_ms']).intersection(set(df2['open_time_ms']))
    
    # Filter both DataFrames to only include rows with common open_times
    df1_common = df1.filter(pl.col('open_time_ms').is_in(common_open_times))
    df2_common = df2.filter(pl.col('open_time_ms').is_in(common_open_times))
    
    # Sort both DataFrames by open_time
    df1_common = df1_common.sort('open_time_ms')
    df2_common = df2_common.sort('open_time_ms')
    
    # Convert to pandas DataFrames
    pdf1 = df1_common.to_pandas()
    pdf2 = df2_common.to_pandas()
    
    # Set open_time as index
    pdf1.set_index('open_time_ms', inplace=True)
    pdf2.set_index('open_time_ms', inplace=True)
    
    # Ensure both DataFrames have the same columns
    columns = set(pdf1.columns).intersection(set(pdf2.columns))
    pdf1 = pdf1[list(columns)]
    pdf2 = pdf2[list(columns)]

    # Round float columns to specified decimal places
    for column in columns:
        if pdf1[column].dtype == 'float64':
            pdf1[column] = pdf1[column].round(decimal_places)
            pdf2[column] = pdf2[column].round(decimal_places)
    
    # Compare DataFrames
    comparison = pdf1 == pdf2
    
    # Find differences
    differences = []
    for column in columns:
        mismatch = ~comparison[column]
        if mismatch.any():
            mismatch_indices = mismatch[mismatch].index
            for idx in mismatch_indices:
                differences.append({
                    'open_time_ms': idx,
                    'column': column,
                    'value1': pdf1.loc[idx, column],
                    'value2': pdf2.loc[idx, column]
                })
    
    return differences

# Example usage:
# differences = compare_dataframes(not1, not2, 'NOTUSDT')
# for diff in differences:
#     print(f"Mismatch at {diff['open_time_ms']} in column {diff['column']}: {diff['value1']} != {diff['value2']}")

In [ ]:
# compare_dataframes (not1_common, not2_common, 'NOTUSDT', decimal_places=2)
compare_dataframes (not1, not2, 'NOTUSDT', decimal_places=2)

In [ ]:
from alpha101 import delta, delay, rank, ts_sum, alpha30

In [ ]:
import pandas as pd
import numpy as np

# Define the symbols
symbols = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA"]

# Define the open_times
open_times = pd.date_range(start="2023-10-01", periods=25, freq="D")

# Create a DataFrame with random integers between 0 and 20
vol_df = pd.DataFrame(
    np.random.randint(0, 5, size=(len(open_times), len(symbols))),
    index=open_times,
    columns=symbols,
)
close_df = pd.DataFrame(
    np.random.randint(0, 21, size=(len(open_times), len(symbols))),
    index=open_times,
    columns=symbols,
)
vol_df
close_df

In [ ]:
vol_df.index

In [ ]:
delta_close = delta (close_df, 1)
delta_close

In [ ]:
rank (delta_close)

In [ ]:
delay (delta_close, 1)

In [ ]:
vol_df

In [ ]:
from alpha101 import delta, delay, rank, ts_sum, correlation, sma

In [ ]:
x = correlation (close_df, vol_df, 5).fillna (0)
sma (delay (close_df, 5), 10)
rank (sma (delay (close_df, 5), 10)) * x
ts_sum (close_df, 5)
t = correlation (ts_sum (close_df, 5), ts_sum (close_df, 10), 4)
t, rank (t)
# close_df, ts_sum (close_df, 3), ts_sum (close_df, 6), correlation (ts_sum (close_df, 3), ts_sum (close_df, 6), 2)

In [ ]:
ts_sum (vol_df, 2)

In [ ]:
import polars as pl
path = 'data/all_data_1d_boris_converted_with_amihud.parquet'
data = pl.read_parquet(path)
data

In [ ]:
alpha30 (close_df, vol_df)

In [ ]:

alpha30 (close_df[1:], vol_df[1:])

In [ ]:
# vol_df, vol_df.cumsum()
vol_df, close_df, correlation (vol_df, close_df, 5)
rank (vol_df), rank (close_df), correlation (rank (vol_df), rank (close_df), 5)

In [ ]:
import polars as pl

# Create a DataFrame
df = pl.DataFrame({
    "key": ["a", "b", "c"],
    "value": [1, 2, 3]
})

# Select the columns and convert to a dictionary
result_dict = df.select([pl.col("key"), pl.col("value")]).to_dict()
result_dict = df.select([pl.col("key"), pl.col("value")]).to_dicts()[0]

print(result_dict)
result_dict['a']


In [ ]:
import polars as pl

# Create a DataFrame
df = pl.DataFrame({
    "key": ["a", "b", "c"],
    "value": [1, 2, 3]
})

# Convert the DataFrame to a list of dictionaries
dict_list = df.to_dicts()

# Extract the keys and values into a dictionary
result_dict = {item['key']: item['value'] for item in dict_list}

print(result_dict)
result_dict['a']
df

In [ ]:
df = pl.DataFrame({
    "a": [4, 5, 6],
    "b": [1, 2, 3],
    "c": [7, 8, 9]
})
second = pl.DataFrame({
    "b" : [2],
    "a" : [1],
    "c" : [3]
})
print (df, second)
df * second

In [ ]:
a = [1]
b = [2]
a+b

In [ ]:
import requests

def fetch_min_trade_notional():
    url = "https://fapi.binance.com/fapi/v1/exchangeInfo"

    try:
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()

        min_trade_notional = {}
        for symbol_info in data['symbols']:
            symbol = symbol_info['symbol']
            for filter_info in symbol_info['filters']:
                if filter_info['filterType'] == 'MIN_NOTIONAL':
                    min_trade_notional[symbol] = filter_info['notional']

        return min_trade_notional

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Binance API: {e}")
        return None

min_trade_values = fetch_min_trade_notional()
if min_trade_values:
    for symbol, notional in min_trade_values.items():
        if int (notional) > 6:
            print(f"Symbol: {symbol}, Min Trade Notional: {notional}")

In [ ]:
import polars as pl

# DataFrame A
df_A = pl.DataFrame({
    "open_time": ["2023-10-01 10:00", "2023-10-01 10:00", "2023-10-01 11:00", "2023-10-01 11:00"],
    "symbol": ["btcusdt", "ethusdt", "btcusdt", "ethusdt"]
})

# DataFrame B
df_B = pl.DataFrame({
    "open_time": ["2023-10-01 10:00", "2023-10-01 11:00"],
    "scale": [1.5, 2.0]
})

df_A, df_B

In [ ]:
# Perform a left join on 'open_time'
df_joined = df_A.join(df_B, on="open_time", how="left")

print(df_joined)

In [ ]:
import polars as pl
path = 'production/all_data_1d_boris_converted_compound_prod.parquet'
data = pl.read_parquet (path).sort ('open_time')
data = data.with_columns(
        pl.from_epoch(pl.col("open_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("open_time"),
        pl.from_epoch(pl.col("close_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("close_time"),
    )
data

In [ ]:
import polars as pl

data = pl.read_parquet ('production/data/all_data_1d_Oct13.parquet')
data

In [ ]:
new_data = pl.read_parquet ('production/data/all_data_1d_Oct19.parquet')
new_data

In [ ]:
new_data.filter (pl.col ('symbol') == 'RLCUSDT')

In [ ]:
import csv
import io

# Step 1: Copy the CSV data into a string variable
csv_data = """symbol,transaction_time,price,side,quantity
RONINUSDT,1729036951019,0.0,BUY,12.5
BADGERUSDT,1729036951070,0.0,BUY,6.0
ONTUSDT,1729036951125,0.0,BUY,101.5
RVNUSDT,1729036951210,0.0,BUY,1158.0
DENTUSDT,1729036951263,0.0,BUY,22075.0
DASHUSDT,1729036951313,0.0,BUY,0.826
DODOXUSDT,1729036951368,0.0,BUY,164.0
CHESSUSDT,1729036951417,0.0,BUY,140.0
BICOUSDT,1729036951462,0.0,BUY,91.0
ALPHAUSDT,1729036951514,0.0,BUY,337.0
XMRUSDT,1729036951568,0.0,BUY,0.13
HFTUSDT,1729036951617,0.0,BUY,129.0
ATAUSDT,1729036951674,0.0,BUY,236.0
LSKUSDT,1729036951723,0.0,BUY,23.0
XVGUSDT,1729036951789,0.0,BUY,4991.0
PERPUSDT,1729036951836,0.0,BUY,31.5
WAXPUSDT,1729036951889,0.0,BUY,579.0
NKNUSDT,1729036951941,0.0,BUY,273.0
STEEMUSDT,1729036951988,0.0,BUY,106.0
DEFIUSDT,1729036952034,0.0,BUY,0.028
ORDIUSDT,1729036952081,-0.0,SELL,0.5"""

short_str = """ENAUSDT,1729036952133,-0.0,SELL,46.0
TIAUSDT,1729036952177,-0.0,SELL,3.0
ETHFIUSDT,1729036952225,-0.0,SELL,11.2
BNBUSDT,1729036952281,-0.0,SELL,0.03
WLDUSDT,1729036952330,-0.0,SELL,8.0
CATIUSDT,1729036952379,-0.0,SELL,45.0
JTOUSDT,1729036952493,-0.0,SELL,8.0
JUPUSDT,1729036952549,-0.0,SELL,22.0"""

# Step 2: Use io.StringIO to simulate a file object
csv_file = io.StringIO(csv_data)

# Step 3: Parse the CSV data
reader = csv.DictReader(csv_file)

# Step 4: Convert the CSV data into a list of dictionaries
data = [row['symbol'] for row in reader]

# Step 5: Print the data
# for row in data:
#     print(row)
data

In [ ]:
mine = ['XMRUSDT', 'COMBOUSDT', 'BICOUSDT', 'GHSTUSDT', 'RVNUSDT', 'DASHUSDT', 'BADGERUSDT', 'BNTUSDT', 'DENTUSDT', 'RLCUSDT', 'ALPHAUSDT', 'CHESSUSDT', 'HFTUSDT', 'ATAUSDT', 'LSKUSDT', 'DODOXUSDT', 'PERPUSDT', 'XVGUSDT', 'WAXPUSDT', 'NKNUSDT', 'STEEMUSDT', 'DEFIUSDT']
mine = mine[2:]
mine

In [ ]:
# set (mine + data) - set(data)
# set (data) - set (mine[2:])
set (data) - set (mine)
set (mine) - set (data) 

In [ ]:
def CoverTimestampToDatetime(data):
    data = data.with_columns(
        pl.from_epoch(pl.col("open_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("open_time"),
        pl.from_epoch(pl.col("close_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("close_time"),
    )
    return data

In [ ]:
import polars as pl
pred = pl.read_parquet ('production/data/predictions_boris_Oct20.parquet')
pred = pl.read_parquet ('production/data/predictions_boris_Oct21.parquet')
pred = CoverTimestampToDatetime (pred).sort ('open_time').select ('symbol', 'open_time', 'close_time', 'linear_compound_factor_7day')
pred = pred.filter (pl.col ('symbol').str.ends_with ('USDT'))
pred

In [ ]:
sorted = pred.filter (pl.col ('open_time') >= pl.datetime (2024, 10, 16)).filter (
        pl.col ('open_time') < pl.datetime (2024, 10, 17)
    ).sort ('linear_compound_factor_7day')
sorted.head (10)

In [ ]:
sorted.tail (10)

In [ ]:
import polars as pl

In [ ]:
LINEAR_INPUT_FILE = f'production/all_data_1d_boris_converted_compound_prod.parquet'
data = pl.read_parquet (LINEAR_INPUT_FILE)
data = data.with_columns(
        pl.from_epoch(pl.col("open_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("open_time"),
        pl.from_epoch(pl.col("close_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("close_time"),
    )


data

In [ ]:
data.columns
data.filter (pl.col ('symbol') == 'NOTUSDT')

In [ ]:
data.filter (pl.col ('symbol') == 'BTCUSDT').sort ('open_time').head (15)

In [ ]:
import pandas as pd

In [ ]:
l = [[1, 3, 2], [3, 4, 2]]
l = [[True, False], [True, True]]
df = pd.DataFrame (l)
df

In [ ]:
df.all()

In [ ]:
for i in df.columns:
    print (i, df[i].all())

In [ ]:
def rank(df):
    if isinstance(df, pd.DataFrame):
        # 如果是DataFrame，对每一行进行rank
        # print (f'rank for pd: {df.shape} == {df}')
        return df.rank(axis=1)
        return df.rank(pct=True, axis=1)
    else:
        assert type(df) == pd.Series
        # 如果是Series，对整个Series进行rank
        # print (f'rank for series: {df.shape} == {df}')
        return df.rank(pct=True)



In [ ]:

rank (df)

In [ ]:
import polars as pl

data = pl.read_parquet ('data/predictions_20241201.parquet')
data.columns

In [ ]:
def print_data (data, symbol):
    data = data.sort (['symbol', 'open_time'])
    data = data.with_columns(
        pl.from_epoch(pl.col("open_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("open_time"),
        pl.from_epoch(pl.col("close_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("close_time"),
    )


    data = data.select (pl.col ([
        'symbol', 'open_time', 'close_time', 'open', 'close', 'past_1day_close_return', 'volume'
    ]))
    if symbol != None:
        data = data.filter (
            pl.col ('symbol') == symbol
        )
    return data

In [ ]:
print_data (data, 'BTCUSDT')

In [ ]:

file2_path = "/home/dp/backup_crypto/crypto/production/z790_predictions.parquet"
old_data = pl.read_parquet (file2_path)


In [ ]:
print_data (old_data, 'BTCUSDT')

In [ ]:
import pandas as pd
df = pd.read_parquet (file2_path)
df

In [ ]:
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df

In [ ]:
delist_data = pl.read_csv ('delist_info.csv')
delist_data

In [ ]:
file1_path = "/home/dp/backup_crypto/crypto/data/predictions_20241201.parquet"
boris = pl.read_parquet (file1_path)
boris

In [ ]:
boris.filter (pl.col ('symbol') == 'BTCUSDT')

In [12]:

import pandas as pd

def convert_to_datetime (df):
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    return df

file1_path = "/home/dp/backup_crypto/crypto/data/predictions_20241129.parquet"
file2_path = "/home/dp/backup_crypto/crypto/production/z790_pred.parquet"

df1 = pd.read_parquet (file1_path)
df2 = pd.read_parquet (file2_path)

df1 = convert_to_datetime (df1)
df2 = convert_to_datetime (df2)

In [13]:
df1.iloc[8246:8249]
df1.dtypes

symbol                             object
open_time                  datetime64[ns]
open                              float64
high                              float64
low                               float64
                                ...      
short_value_scale_8day            float64
long_value_scale_9day             float64
short_value_scale_9day            float64
long_value_scale_10day            float64
short_value_scale_10day           float64
Length: 108, dtype: object

In [14]:
df1[(df1['symbol'] == 'AGIXUSDT') & (df1['open_time'] >= pd.to_datetime('2024-05-29 00:00:00'))]
df1[(df1['symbol'] == 'AGIXUSDT')]

,symbol,open_time,open,high,low,close,volume,close_time,quote_volume,count,...,long_value_scale_6day,short_value_scale_6day,long_value_scale_7day,short_value_scale_7day,long_value_scale_8day,short_value_scale_8day,long_value_scale_9day,short_value_scale_9day,long_value_scale_10day,short_value_scale_10day
7944,AGIXUSDT,2023-02-16,0.4465,0.4700,0.3978,0.4044,38639133.0,2023-02-16 23:59:59.999,1.701465e+07,88287,...,1.2,0.8,1.2,0.8,1.0,1.0,0.8,1.2,1.2,0.8
7945,AGIXUSDT,2023-02-17,0.4045,0.4460,0.4028,0.4374,82911909.0,2023-02-17 23:59:59.999,3.585506e+07,208173,...,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.0,1.0
7946,AGIXUSDT,2023-02-18,0.4376,0.4633,0.3900,0.4200,70503952.0,2023-02-18 23:59:59.999,3.067230e+07,155053,...,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8
7947,AGIXUSDT,2023-02-19,0.4199,0.4321,0.3910,0.4005,72165008.0,2023-02-19 23:59:59.999,2.996534e+07,156308,...,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8
7948,AGIXUSDT,2023-02-20,0.4005,0.4110,0.3774,0.3892,94144937.0,2023-02-20 23:59:59.999,3.691557e+07,168579,...,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8435,AGIXUSDT,2024-06-21,0.6559,0.6729,0.6355,0.6624,147116079.0,2024-06-21 23:59:59.999,9.594593e+07,558950,...,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2
8436,AGIXUSDT,2024-06-22,0.6625,0.6655,0.6127,0.6188,100049706.0,2024-06-22 23:59:59.999,6.376706e+07,356609,...,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2
8437,AGIXUSDT,2024-06-23,0.6187,0.6338,0.5880,0.5908,94825492.0,2024-06-23 23:59:59.999,5.795263e+07,307297,...,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2
8438,AGIXUSDT,2024-06-24,0.5908,0.6922,0.5603,0.6688,166368284.0,2024-06-24 23:59:59.999,1.015212e+08,559589,...,1.0,1.0,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2


In [15]:

df2[df2['symbol'] == 'AGIXUSDT']
df2[(df2['symbol'] == 'AGIXUSDT') & (df2['open_time'] >= pd.to_datetime('2024-05-29 00:00:00'))]

,symbol,open_time,open,high,low,close,volume,close_time,quote_volume,count,...,long_value_scale_6day,short_value_scale_6day,long_value_scale_7day,short_value_scale_7day,long_value_scale_8day,short_value_scale_8day,long_value_scale_9day,short_value_scale_9day,long_value_scale_10day,short_value_scale_10day
8546,AGIXUSDT,2024-05-29,0.9348,0.9768,0.9139,0.9181,63506607.0,2024-05-29 23:59:59.999,5.999269e+07,344947,...,1.2,0.8,1.2,0.8,1.2,0.8,1.0,1.0,1.2,0.8
8547,AGIXUSDT,2024-05-30,0.9180,0.9582,0.8840,0.9135,76233912.0,2024-05-30 23:59:59.999,7.057392e+07,380616,...,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.0,1.0
8548,AGIXUSDT,2024-05-31,0.9135,0.9260,0.8758,0.8971,55920763.0,2024-05-31 23:59:59.999,5.050518e+07,283273,...,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8
8549,AGIXUSDT,2024-06-01,0.8970,0.8999,0.8797,0.8927,22705044.0,2024-06-01 23:59:59.999,2.018915e+07,138314,...,1.2,0.8,1.2,0.8,1.0,1.0,1.2,0.8,1.2,0.8
8550,AGIXUSDT,2024-06-02,0.8927,0.9043,0.8581,0.8772,28849474.0,2024-06-02 23:59:59.999,2.549773e+07,169752,...,1.0,1.0,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8
8551,AGIXUSDT,2024-06-03,0.8772,0.9148,0.8655,0.8893,43787478.0,2024-06-03 23:59:59.999,3.929032e+07,221286,...,0.8,1.2,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8
8552,AGIXUSDT,2024-06-04,0.8893,0.9041,0.8711,0.8967,36128266.0,2024-06-04 23:59:59.999,3.208160e+07,171590,...,1.2,0.8,1.0,1.0,1.2,0.8,1.2,0.8,1.2,0.8
8553,AGIXUSDT,2024-06-05,0.8968,0.9322,0.8968,0.9079,37468787.0,2024-06-05 23:59:59.999,3.421480e+07,190957,...,1.2,0.8,1.0,1.0,1.2,0.8,1.2,0.8,1.2,0.8
8554,AGIXUSDT,2024-06-06,0.9079,0.9132,0.8489,0.8635,40391318.0,2024-06-06 23:59:59.999,3.580636e+07,199805,...,1.2,0.8,1.2,0.8,1.0,1.0,1.2,0.8,1.0,1.0
8555,AGIXUSDT,2024-06-07,0.8635,0.8715,0.6689,0.7762,105376271.0,2024-06-07 23:59:59.999,8.341380e+07,419517,...,1.0,1.0,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2
